# Download TrendMiner Context-Item Attachments

The script is designed to automate the retrieval of documents or files attached to specific context items within the TrendMiner platform.  
Context items in TrendMiner are annotations or events linked to process data, often enriched with additional information like attachments.  
By executing this script, you can programmatically download attachments associated with context items without manually navigating through the TrendMiner interface.  

> **Tip:**  
> To use this script you will need the **short key** of the context item (for example `386-3` in the screenshot), which appears as the title when you open a context item. You can also inspect the **Attachments** tab to see available attachment names, which you can use in the script.
![Context Item Example](https://userguide.trendminer.com/image/uuid-ba68e2ee-97c6-7b28-95b0-a9138c3d6623.png)

In [ ]:
import os
import requests
from typing import Optional, Dict

def get_context_attachments(
    key: str,
    attachment_name: Optional[str] = None,
) -> Dict[str, bytes]:
    """
    Fetch one or all attachments from a TrendMiner context item.

    :param key:                  the context-item key (e.g. "FBC-9C")
    :param attachment_name:      if given, only download that attachment; else download all
    :return:                     dict mapping attachment names to raw bytes
    :raises FileNotFoundError:   if attachment_name was specified but not found
    """
    # fallback to environment
    server = os.getenv("KERNEL_SERVER_URL", "").rstrip('/')
    bearer = os.getenv("KERNEL_USER_TOKEN", "")
    if not server or not bearer:
        raise ValueError("Must provide server_url and token (or set KERNEL_SERVER_URL and KERNEL_USER_TOKEN)")

    sess = requests.Session()
    sess.verify = False
    sess.headers.update({"Authorization": f"Bearer {bearer}"})

    # 1) get the context‐item metadata
    item_resp = sess.get(f"{server}/context/item/{key}")
    item_resp.raise_for_status()
    item = item_resp.json()
    context_id = item["identifier"]

    # 2) find the attachments link
    attachments_url = next(
        link["href"] for link in item.get("links", [])
        if link.get("rel") == "attachments" or "attachment" in link.get("href", "")
    )

    # 3) list attachments
    list_resp = sess.get(attachments_url)
    list_resp.raise_for_status()
    attachments = list_resp.json().get("content", [])

    results: Dict[str, bytes] = {}
    for att in attachments:
        name = att["name"]
        if attachment_name and name != attachment_name:
            continue

        att_id = att["identifier"]
        dl_resp = sess.get(f"{server}/context/data/{context_id}/attachments/{att_id}/download")
        dl_resp.raise_for_status()
        results[name] = dl_resp.content

        if attachment_name:
            break

    if attachment_name and not results:
        raise FileNotFoundError(f"No attachment named {attachment_name!r} in context item {key!r}")

    return results

In [ ]:
# Fetch all attachments
all_atts = get_context_attachments("386-3")
print(all_atts)

# Fetch just one by name
one_att = get_context_attachments("FBC-9C", attachment_name="import_name")
print(one_att["import-2.0-example"])